# Inference 

In [1]:
import sys
sys.path.append('/kaggle/input/specs-fr-eegs')
from ResNet1d_GRU_hybrid import *

0it [00:00, ?it/s]

In [2]:
#pop1 CV 0.4588: LB 0.56
#model_weights = [x for x in glob('/kaggle/input/hms-resnet1d-gru-train/pop_1_weight_oof/*.pth')]
#CV 0.51 
model_weights = [x for x in glob('/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/*.pth')]
model_weights

['/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/resnet1d_gru_fold0_best_version9_stage2.pth',
 '/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/resnet1d_gru_fold1_best_version9_stage2.pth',
 '/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/resnet1d_gru_fold4_best_version9_stage2.pth',
 '/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/resnet1d_gru_fold3_best_version9_stage2.pth',
 '/kaggle/input/hms-resnet1d-gru-train/pop_2_weight_oof/resnet1d_gru_fold2_best_version9_stage2.pth']

In [3]:
predictions = []

for model_weight in model_weights:
    test_dataset = EEGDataset(test_df, CFG, mode='test')
    train_loader = DataLoader(
        test_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=False
    )
    model = EEGNet(kernels=[3,5,7,9,11], in_channels=CFG.in_channels, fixed_kernel_size=5, num_classes=CFG.target_size)
    checkpoint = torch.load(model_weight, map_location=device)
    model.load_state_dict(checkpoint["model"])
    model.to(device)
    prediction_dict = inference_function(test_loader, model, device)
    predictions.append(prediction_dict["predictions"])
    torch.cuda.empty_cache()
    gc.collect()
    
predictions = np.array(predictions)
pred_M6 = np.mean(predictions, axis=0)

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

# Submission

In [4]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
sub = pd.DataFrame({'eeg_id': test_df.eeg_id.values})
sub[TARGETS] = pred_M6
sub.to_csv(f'submission.csv',index=False)
print(f'Submission shape: {sub.shape}')
sub.head()

Submission shape: (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.035991,0.058254,0.024876,0.120533,0.10352,0.656826
